In [1]:
import torch
import torch.nn as nn
from vae_models import VAE_CONV_NeuralModel
from mnist_classifier import NeuralModel
from classifier_attacks import test 
from utils import data_loader
import torch.nn.functional as F

In [2]:
train_set, test_set = data_loader.get_data()

batch_size = 1

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [ ]:
class VAEClassifierModel(nn.Module, beta):
    def __init__(self):
        super().__init__()

        self.vae = VAE_CONV_NeuralModel()
        self.vae.load_state_dict(torch.load("models/trained_CONV_vae_B="+str(beta)))

        self.classification_model = NeuralModel()
        self.classification_model.load_state_dict(torch.load("models/trained_model"))

    def forward(self, x):
        after_vae = self.vae(x)
        classification = self.classification_model(after_vae[0])

        return classification
        pass